In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 100000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('data/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[65, 68, 76,  1, 66, 68, 54, 67],
        [58, 78, 58, 65, 62, 57, 72,  1],
        [58, 72, 58,  1, 76, 62, 67, 60],
        [68, 76, 67,  1, 73, 61, 58,  1]], device='cuda:0')
targets:
tensor([[68, 76,  1, 66, 68, 54, 67,  0],
        [78, 58, 65, 62, 57, 72,  1, 68],
        [72, 58,  1, 76, 62, 67, 60, 72],
        [76, 67,  1, 73, 61, 58,  1, 72]], device='cuda:0')


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



VXBPzLtUQ JEOMHO5CxoAuq2MyG?VL""k2﻿zf*or'zLtf;gFs)zArT1Ij1MIV5tOu8Jix;Z ﻿j-'RPwhAkO'6i0U(0Kf_JF'U3UIB
-p*u"&M3y1suVy﻿lxwo:11AzGeQaT9R2LOSf)S[(2RZ(7Gk08tgjG)j]ME8q]L:
j&sduf)slP-O5u5)vKxF"Sk,h*(f:Mj-)n76.''sCKVP:PJ4ZhR!p!X)s)j&p"JY1LrSqYMI!*i3gEkb21D16!KBP.(3h0DDS89T08﻿ J?s3JOlfbCg[(o
24;ydi'V﻿pW3brTlw[Z1Z sTbq﻿tl0YmIVJ﻿mNnU8:2t(N7TA8
[nQ),DxlGqBO7!JPRQmB9z'U25y_
nuUP:uzDjc]Qct?"yfQW.SS:&y!"nz''.tf00blRcY",fknjueH1c8)sW8D1s;[(?LyKa[ x;:kaTBc6n]"tT-(M;Jj!1p?!qJj?3XG)c9(h&VDARo5wn"tkHO]mBZE7QbKv2md


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.867, val loss: 4.881
step: 250, train loss: 4.819, val loss: 4.796
step: 500, train loss: 4.747, val loss: 4.753
step: 750, train loss: 4.675, val loss: 4.682
step: 1000, train loss: 4.632, val loss: 4.629
step: 1250, train loss: 4.571, val loss: 4.574
step: 1500, train loss: 4.492, val loss: 4.511
step: 1750, train loss: 4.440, val loss: 4.452
step: 2000, train loss: 4.396, val loss: 4.422
step: 2250, train loss: 4.360, val loss: 4.334
step: 2500, train loss: 4.293, val loss: 4.305
step: 2750, train loss: 4.243, val loss: 4.254
step: 3000, train loss: 4.193, val loss: 4.195
step: 3250, train loss: 4.144, val loss: 4.141
step: 3500, train loss: 4.081, val loss: 4.087
step: 3750, train loss: 4.027, val loss: 4.044
step: 4000, train loss: 4.000, val loss: 4.001
step: 4250, train loss: 3.946, val loss: 3.952
step: 4500, train loss: 3.900, val loss: 3.904
step: 4750, train loss: 3.852, val loss: 3.846
step: 5000, train loss: 3.828, val loss: 3.841
step: 5250, train l

***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)